In [9]:
# alternate method to get keywords. LARA method based on Chi-square is basically same, but much better
def getKeywords(data = data, freq_threshold = .5, prob_threshold = 0.2, words_per_iter = 4, iters = 3):

    #break down reviews into sentences and break down each sentence into words using tokenizer and remove stopwords
    # returns list where each item is the list of words in that sentence
    sentence_words = []
    for review in data['Content']:
        review = review.decode('utf-8')
        sentences = nltk.tokenize.sent_tokenize(review)
        for sentence in sentences:
            sentence_words.append([x.lower() for x in nltk.tokenize.word_tokenize(sentence) if x not in stopwords.words('english') and len(x) > 2])    
    
    seeds =  {"Value" : ["value", "price", "quality","worth"],
              "Rooms" : ["room", "suite", "view", "bed"],
              "Location" : ["location", "traffic", "minute", "restaurant"],
              "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
              "Check in / front desk": ["stuff", "check", "help", "reservation"],
              "Service" : ["service", "food", "breakfast", "buffet"],
              "Business service" : ["business", "center", "computer", "internet"]
             }

    # find Probability(sentence(S) has aspect(A) GIVEN S has word(W)) = count(S that have A and have W) / count(S that have W)

    for i in range(iters):
        
        sents_with_word_asp = {}
        sents_with_word = {}
        sents_with_aspect = {}
        prob_asp_given_word = {}

        # calculates counts of (S that have W) and (S that have A and W)
        for sentence in sentence_words:
            for word in sentence:
                sents_with_word[word] = sents_with_word.get(word,0) + 1
                for aspect, aspect_words in seeds.items():
                    for aspect_word in aspect_words:
                        if aspect_word in sentence:
                            sents_with_word_asp[(word,aspect)] = sents_with_word_asp.get((word, aspect), 0) + 1
                            sents_with_aspect[aspect] = sents_with_aspect.get(aspect,0) + 1
                            break

        for (word, aspect), count in sents_with_word_asp.items():
            #susceptible to low frequencies. hence freq_threshold
            #freq_threshold ensures that count(S with  W) is atleast x% of count(S)
            if sents_with_word[word] > (freq_threshold/100.0)*len(sentence_words):
                prob_asp_given_word[(word,aspect)] = count/float(sents_with_word[word])

        prob_asp_given_word_sorted = sorted(prob_asp_given_word.items(), key=itemgetter(1),reverse=True)
        
        for aspect, word_list in seeds.items():
            count = 0
            for item in prob_asp_given_word_sorted:
                #item is of the form ((word,aspect),probability)
                if item[0][1] == aspect:
                    if item[0][0] not in word_list:
                        if count <= words_per_iter:
                            if item[1] >= prob_threshold:
                                seeds[aspect].append(item[0][0])
                                count += 1
                            else:
                                #because sorted, the others can't have higher probability
                                break
                        else:
                            # because limiit of words per aspect in this iteration has been reached
                            break

    return seeds